# Mini-Project 3: Classification
Jimmy Sisenglath, Eric Pham, Justin Eugenio

In [1]:
# Import libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.feature_extraction.text as sk_text

from sklearn import tree # Decision Tree.
from sklearn import metrics
from sklearn.svm import SVC # Support Vector Machine.
from sklearn.preprocessing import OneHotEncoder # One-hot encoding.
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.linear_model import LogisticRegression # Logistic Regression.
from sklearn.preprocessing import StandardScaler # Neural networks.
from sklearn.neural_network import MLPClassifier # Neural networks.
from sklearn.model_selection import cross_val_score # Cross validation.
from sklearn.model_selection import train_test_split, GridSearchCV # Grid search.

# 1. Classification of Twitter Users
- In this project you will use the **original data you used in Project 1**. You will practice with algorithms for classification. **The goal of this project is to create classification models that predict if a user is a follower of Trump or Clinton.** In the file “clinton_trump_user_classes.txt”, we have the ground truth “class” membership for each user id in the data. Class 0 corresponds to Trump followers, while class 1 corresponds to Clinton followers.

In [2]:
# Read the 2016 Presidential election tweets and assigned column names. 

# Produces the tweet column name title on top of the table. Record type.
columns = ['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount','Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount','FavoriteCount', 'Text']

# Old dataframe. Use the original data you used in Project 1.
tweets = pd.read_table('clinton_trump_tweets.txt', header=None, names=columns, encoding="ISO-8859-1")
pd.set_option('display.max_colwidth', -1) # Shows the whole message in the records.

# Prepare ground truth. Predict if a user is a follower of "Trump or Clinton (Supporter)".
# ground_truth = pd.read_table('data/clinton_trump_user_classes.txt', names=["UserID", "Supporter"], dtype=int, encoding ="ISO-8859-1")

# Show first 5 records.
tweets.head()

,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,Cookie,Cookiemuffen,109945090,2154,2034,The American South,Got married after college. I don't regret starting a family instead of grad school. Proud Deplorable,Wed Oct 26 18:44:08 EEST 2016,791304413923213312,en,NaN,1937,0,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,Mark Hager,marksnark,167177185,204,448,Pittsburgh,"Hip, trendy, smart, funny, fit, lobbyist. U? Boilerplate: these thoughts are my own, not anyone else's. Hmmmkay?",Wed Oct 26 00:33:20 EEST 2016,791029904733331457,en,NaN,891,0,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,Capitalist Creations,aaronjhoddinott,1191022351,775,154,Canada,"Entrepreneur, startup investor, political junkie, free market supporter, beer connoisseur, dad and dog lover. Also a golf enthusiast despite my lack of skill.",Fri Oct 28 05:05:10 EEST 2016,791823089700962304,en,NaN,7,0,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.


In [3]:
# Drop unused columns.
tweets = tweets.drop(columns=['Name','ScreenName','FollowersCount','FriendsCount', 'Description','CreatedAt','StatusID','Language','Place','RetweetCount','FavoriteCount'])

In [4]:
# Drop null values in 'Location'.
# dropna: remove missing values.
tweets = tweets.dropna(subset=['Location'])

# Task 1.1
- Remove all retweets first. Remove all users that have less than 20 tweets. You may want to keep the entire tweet content, including hashtags/handles. For the remaining users, use **all available information in the dataset that you consider useful to create features** for classification (such as **Location, Description, Place**). You are also encouraged to use any conclusions you draw in Project 2 (clustering) to create any features to improve the classification result. **Use train_test_split() to split data into training and test sets, where 20 percent of the records go to test set.**

## *Remove all retweets first.*

In [5]:
tweets = tweets.drop(tweets[(tweets.Text.str.startswith("RT"))].index)

tweets.head()

,UserID,Location,Text
0,1519696717,"Little Rock, Arkansas",@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
6,106568768,"Dallas, TX",#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,17101060,The Universe,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,16818809,Long Island and beyond...,Hulk smash!
12,163712593,"Portland, Oregon","Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"


## *Remove all users that have less than 20 tweets. You may want to keep the entire tweet content, including hashtags/handles.*

In [6]:
# Use group by and count to retrive amounts of tweets for each individual UserID from database.
top_users = tweets.groupby('UserID').Text.count().sort_values(ascending=False)

# Include users with >= 20 tweets.
top_users = top_users[top_users>=20]

top_users.head()

UserID
1971604699    3018
15501305      2493
17197477      2321
1732989216    2210
23719043      2133
Name: Text, dtype: int64

In [7]:
# Refresh dataframe with UserID that have 20 tweets or more.
# Using isin, if the userID is in the 'top_users' list.
tweets =  tweets[tweets['UserID'].isin(top_users.index.tolist())]

tweets.head()

,UserID,Location,Text
0,1519696717,"Little Rock, Arkansas",@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
6,106568768,"Dallas, TX",#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,17101060,The Universe,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,16818809,Long Island and beyond...,Hulk smash!
12,163712593,"Portland, Oregon","Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"


In [8]:
new_tweets = tweets.groupby(['UserID',"Location"], as_index=False).sum()

new_tweets.head()

UserID               Location  \
0  1737    In my head              
1  2391    San Francisco           
2  2426    SF                      
3  11569   Jet Life Stadium        
4  14763   Portland, Oregon, USA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [9]:
# For ground truth. 
tweet_cols2 = ['UserID', 'Supporter']
ground_truth = pd.read_table('clinton_trump_user_classes.txt', names=tweet_cols2, encoding ="ISO-8859-1")

In [10]:
# Show 'Supporter' column by merge.
merge_tweets = pd.merge(new_tweets, ground_truth, on='UserID')

merge_tweets.head()

UserID               Location  \
0  1737    In my head              
1  2391    San Francisco           
2  2426    SF                      
3  11569   Jet Life Stadium        
4  14763   Portland, Oregon, USA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [11]:
# Dataframe hashtag aggregate. 

# To obtain vector representation of each user, you may group all the 
# hashtags/handles by each user and then create a dataframe where each 
# line has a user and all her hashtags/handles. Then 
# call tfidfVectorzier() on that dataframe.

vectorizer = sk_text.TfidfVectorizer(
    stop_words='english',
    max_features=1000, # 2000 takes a while...
                                     
    # .1 Means "ignore terms that appear in less than 1% of the documents".
                                     
    # The default min_df is 1, which means "ignore terms that appear in less than 1 document". 
    # Thus, the default setting does not ignore any terms.        
    min_df=500, 
                             
    # If .5: ignore terms that appear in more than 50% of the documents.
    #max_df=.9, 
)

matrix = vectorizer.fit_transform(merge_tweets.Text)
print(type(matrix))         
tfidf_data = matrix.toarray() 
print(tfidf_data)

<class 'scipy.sparse.csr.csr_matrix'>
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.02194481 ... 0.         0.         0.        ]
 ...
 [0.02100724 0.01587684 0.00577047 ... 0.         0.01444637 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [12]:
# Vectorize Location.

vectorizer2 = sk_text.TfidfVectorizer(
    stop_words='english',
    max_features=1000, # Try 2000?
                                     
    # .1 Means "ignore terms that appear in less than 1% of the documents".
                                     
    # The default min_df is 1, which means "ignore terms that appear in less than 1 document". 
    # Thus, the default setting does not ignore any terms.        
    min_df=500, 
                             
    # If .5: ignore terms that appear in more than 50% of the documents.
    #max_df=.9, 
)

matrix2 = vectorizer2.fit_transform(merge_tweets.Location)
print(type(matrix2))         
tfidf_data2 = matrix2.toarray() 
print(tfidf_data2)

<class 'scipy.sparse.csr.csr_matrix'>
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [13]:
count_vect_df = pd.DataFrame(matrix.todense(), columns=vectorizer.get_feature_names())

# Location count vectorizer.
count_vect_df2 = pd.DataFrame(matrix2.todense(), columns=vectorizer2.get_feature_names())

merge_tweets = pd.concat([merge_tweets, count_vect_df], axis=1)
merge_tweets = pd.concat([merge_tweets, count_vect_df2], axis=1)

## *Use train_test_split() to split data into training and test sets, where 20 percent of the records go to test set.*

In [14]:
# Dataframe with hashtags & handles & ground truth.
X = merge_tweets.drop(columns=['UserID','Text','Location','Supporter'])
# Trump or Clinton 'Supporter' values.
y = merge_tweets['Supporter'].values

In [15]:
# Shapes show.
print(X.shape)
print(y.shape)

# 19515 average in total and split 19515 records to 2 parts.

(19515, 1005)
(19515,)


In [16]:
# Split dataset into train and test data.
# Predict. Test the size. How many records you want to go to test data. 20% of the records will go to test data (.2)
# and remaining to to other set. Specify the first 3 parameter correctly.


# random_state: is used for initializing the internal random number generator, which will 
# decide the splitting of data into train and test indices.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4, stratify=y)

# Task 1.2
- **Train Decision Tree, SVM, Logistic Regression, and Neural Networks.** In your report describe the features that you used for each classifier.

## *Decision Tree*

In [17]:
dtree = tree.DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)

# Print out the model.
dtree.fit(X_train, y_train)

# These make predictions on the testing set... 
# Apply model on X_test.
#y_pred = dtree.predict(X_test)

#print("Confusion Matrix:")
#print(metrics.confusion_matrix(y_test, y_pred))
#print('Precision Score:', metrics.precision_score(y_test, y_pred, average= 'weighted'))
#print('Recall Score:', metrics.recall_score(y_test, y_pred, average= 'weighted'))
#print('F1 Score:', metrics.f1_score(y_test, y_pred, average= 'weighted'))            

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## *SVM*

In [18]:
# Probability: boolean, optional (default=False)  
# Whether to enable probability estimates.

# 2 most important parameters:
# C: float, optional (default=1.0)
# Setting gamma used in kernel to ‘auto’ will use 1 / n features

# Use support vector machine. C and gamma. 
# C: how much penalty to give to misclassfied records.

scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

# 700 got 4min 53s.
clf = SVC(C=1.0, gamma='auto', cache_size=700)

# Print out the model.
clf.fit(X_train, y_train)

#y_pred = clf.predict(X_test)

#print("Confusion Matrix:")
#print(metrics.confusion_matrix(y_test, y_pred)) 
#print('Precision Score:', metrics.precision_score(y_test, y_pred, average= 'weighted'))
#print('Recall Score:', metrics.recall_score(y_test, y_pred, average= 'weighted'))
#print('F1 Score:', metrics.f1_score(y_test, y_pred, average= 'weighted'))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## *Logistic Regression*

In [19]:
# Set a few optional parameter.
lr = LogisticRegression(solver='lbfgs', # Optimized waiting descent.
                        max_iter=200, 
                        multi_class='auto') 

# Print out the model.
lr.fit(X_train, y_train)

#y_pred = lr.predict(X_test)

#print("Confusion Matrix:")
#print(metrics.confusion_matrix(y_test, y_pred)) 
#print('Precision Score:', metrics.precision_score(y_test, y_pred, average= 'weighted'))
#print('Recall Score:', metrics.recall_score(y_test, y_pred, average= 'weighted'))
#print('F1 Score:', metrics.f1_score(y_test, y_pred, average= 'weighted'))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

## *Neural Networks*

In [20]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Multi-layer Perception Model (mlp)
# hidden_layer_sizes: 3 hidden layers, each has 30 neurons.
# 10, 20, 30, or 100... as long as your machine can handle that. Keep as many layer as you want.

mlp = MLPClassifier(hidden_layer_sizes=(30,30,30), 
                    solver='adam', # Adam optimization algo. Variation of gradient descent.
                    max_iter=1000) # Create neural network with hidden layers. Go up to 1000 iterations. 

# Print out the model.
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [21]:
# Output report.
# print(metrics.classification_report(y_test, y_pred))

#print("Confusion Matrix:")
#print(metrics.confusion_matrix(y_test, y_pred, labels=[0, 1]))
#print('Precision Score:', metrics.precision_score(y_test, y_pred, average= 'weighted'))
#print('Recall Score:', metrics.recall_score(y_test, y_pred, average= 'weighted'))
#print('F1 Score:', metrics.f1_score(y_test, y_pred, average= 'weighted'))

# Task 1.3
- **Train k-NN model.** In your report describe the features that you used for k-NN. **Perform parameter tuning on k-NN model. Apply 5-fold cross validation** and use **grid search** to find the best K value for k-NN model. Set scoring metric to F1 score (F-measure). Use the best K value identified from grid search to train your k-NN model. **Plot the F1 score against K value based on the results you achieved from grid search.**

## *Train k-NN Model*

In [22]:
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 12)
# Fit the classifier to the data
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=12, p=2,
           weights='uniform')

In [23]:
knn.score(X_test, y_test)

0.6161926723033564

In [ ]:
%%time
# Instantiate the grid.

# Verbose: Verbosity. Higher it is, the more messages.
# Refit: Estimator using the best found parameters on the whole dataset.

# So, the maximum performance that can be obtain always is using n_jobs = -1. | None is default (1).
grid = GridSearchCV(knn, param_grid, cv=5, n_jobs=2, scoring='f1_weighted') # call grid search CV on my model. 5 fold validation (cv=5).

# Fit the classifier to the data.
# knn.fit(X_train,y_train)

grid.fit(X_train, y_train) # Fit the grid with data.
# train this model on your data. every value here to create a model. specify 49 different kvalue/models.

In [ ]:
# View the complete results (list of named tuples).
means = grid.cv_results_['mean_test_score']

means

In [ ]:
# Plot the results.
plt.plot(range(1,21),means)
plt.xlabel('Value of K for KNN')
plt.ylabel('F1 score based on Cross-Validation')
plt.show()

In [ ]:
# Identify the best model.
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
# Train model using all data and the best known parameters.
# Using best n_neighbors from grid search. Not finished.
knn = KNeighborsClassifier(n_neighbors=1, weights='uniform')
knn.fit(X_train, y_train)
y_pred =  knn.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))

# Task 1.4
- **Using the test set**, compute the confusion matrix, the precision, recall and F-measure for (1) Decision Tree, (2) SVM, (3) Logistic Regression, (4) Neural Networks, and (5) k-NN. For k-NN model, use the best K value identified from grid search. **Compare their performance and include your conclusions in your report**

In [29]:
#Decision Tree
dtree_pred_class = dtree.predict(X_test)

dtreeConfusion = metrics.confusion_matrix(y_test, dtree_pred_class, labels=[0, 1])
dtreeRecall = metrics.recall_score(y_test, dtree_pred_class)
dtreePrecision = metrics.precision_score(y_test, dtree_pred_class)
dtreeF1 = metrics.f1_score(y_test, dtree_pred_class)

#confusion matrix
print("Decision tree confusion matrix:")
print(dtreeConfusion)
print("")
#precision
print("Decision tree precision:")
print(dtreePrecision)
print("")
#recall
print("Decision tree recall:")
print(dtreeRecall)
print("")
#f-measure
print("Decision tree F-measure:")
print(dtreeF1)

Decision tree confusion matrix:
[[ 894  827]
 [1176 1006]]

Decision tree precision:
0.5488270594653574

Decision tree recall:
0.461044912923923

Decision tree F-measure:
0.501120797011208


In [28]:
#SVM
SVM_pred_class = clf.predict(X_test)

SVMconfusion = metrics.confusion_matrix(y_test, SVM_pred_class, labels=[0, 1])
SVMrecall = metrics.recall_score(y_test, SVM_pred_class)
SVMprecision = metrics.precision_score(y_test, SVM_pred_class)
SVMf1 = metrics.f1_score(y_test, SVM_pred_class)

#confusion matrix
print("SVM confusion matrix:")
print(SVMconfusion)
print("")
#precision
print("SVM precision:")
print(SVMprecision)
print("")
#recall
print("SVM recall:")
print(SVMrecall)
print("")
#f-measure
print("SVM F-measure:")
print(SVMf1)

SVM confusion matrix:
[[ 436 1285]
 [  43 2139]]

SVM precision:
0.6247079439252337

SVM recall:
0.9802933088909258

SVM F-measure:
0.7631109525508384


In [30]:
#Logistic Regression
lr_pred_class = lr.predict(X_test)

lrConfusion = metrics.confusion_matrix(y_test, lr_pred_class, labels=[0, 1])
lrRecall = metrics.recall_score(y_test, lr_pred_class)
lrPrecision = metrics.precision_score(y_test, lr_pred_class)
lrF1 = metrics.f1_score(y_test, lr_pred_class)

#confusion matrix
print("Logistic regression confusion matrix:")
print(lrConfusion)
print("")
#precision
print("Logistic regression precision:")
print(lrPrecision)
print("")
#recall
print("Logistic regression recall:")
print(lrRecall)
print("")
#f-measure
print("Logistic regression F-measure:")
print(lrF1)

Logistic regression confusion matrix:
[[1262  459]
 [ 640 1542]]

Logistic regression precision:
0.7706146926536732

Logistic regression recall:
0.7066911090742438

Logistic regression F-measure:
0.7372699019842219


In [31]:
#Neural Networks
mlp_pred_class = mlp.predict(X_test)

mlpConfusion = metrics.confusion_matrix(y_test, mlp_pred_class, labels=[0, 1])
mlpRecall = metrics.recall_score(y_test, mlp_pred_class)
mlpPrecision = metrics.precision_score(y_test, mlp_pred_class)
mlpF1 = metrics.f1_score(y_test, mlp_pred_class)

#confusion matrix
print("Neural Networks confusion matrix:")
print(mlpConfusion)
print("")
#precision
print("Neural Networks precision:")
print(mlpPrecision)
print("")
#recall
print("Neural Networks recall:")
print(mlpRecall)
print("")
#f-measure
print("Neural Networks F-measure:")
print(mlpF1)

Neural Networks confusion matrix:
[[1123  598]
 [ 611 1571]]

Neural Networks precision:
0.7242969110189027

Neural Networks recall:
0.7199816681943172

Neural Networks F-measure:
0.7221328430245921
